## Part 1. ETL Pipeline for pre-processing the files

In [1]:
# import libraries 
import os 
import json 
import glob 
import numpy as np
import pandas as pd 
import re
import cassandra

### Creating list of filepaths to process original event csv data files

In [2]:
# chekcing current directory 
print(os.getcwd())

D:\jupy\Data Modeling with Apache Cassandra


In [3]:
# get dataset path 
filepath = os.getcwd()+'/event_data'

In [4]:
# create a for loop to create a list of files and collect each filepath 
for root, dir, file in os.walk(filepath):
    
    # join the file path and roots with the subdirectory using glob 
    file_path_list = glob.glob(os.path.join(root,'*'))

In [5]:
# checking 
print(file_path_list)

['D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-01-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-02-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-03-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-04-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-05-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-06-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-07-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-08-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-09-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-10-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-11-events.csv', 'D:\\jupy\\Data Modeling with Apache Cassandra/event_data\\2018-11-12-events.csv', 'D:

### Processing the files to create the datafile csv that will be used for Apache Cassandra tables

In [9]:
import csv

In [13]:
# initialize 
full_data_rows_list = []

# for every filepath in the file path list 
for file in file_path_list:
    #reading csv file 
    with open(file, 'r', encoding='utf8', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader)
        
        for line in csv_reader:
            full_data_rows_list.append(line)
            
            

In [17]:
full_data_rows_list[0]

['',
 'Logged In',
 'Walter',
 'M',
 '0',
 'Frye',
 '',
 'free',
 'San Francisco-Oakland-Hayward, CA',
 'GET',
 'Home',
 '1.54092E+12',
 '38',
 '',
 '200',
 '1.54111E+12',
 '39']

In [15]:
# creating a small event data csv file 
csv.register_dialect('myDialect',quoting=csv.QUOTE_ALL,skipinitialspace=True)

In [22]:
with open('event_datafile_new.csv','w',encoding='utf8',newline='') as f:
    csv_writer=csv.writer(f,dialect='myDialect')
    csv_writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                     'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        csv_writer.writerow((row[0],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[12],row[13],row[16]))

In [23]:
# check the number of rows in new csv file 
with open('event_datafile_new.csv','r',encoding='utf8') as f:
    print(sum(1 for line in f))

6821


### Part 2 

#### creating cluster 

In [25]:
from cassandra.cluster import Cluster

In [26]:
# make cluster 
cluster = Cluster()

# establish connection 
session = cluster.connect()

NoHostAvailable: ('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(10061, "Tried connecting to [('127.0.0.1', 9042)]. Last error: 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다")})